In [1]:
import pandas as pd
df_lists = pd.read_csv("C:/python/python_for_data_science/hackason/listings.csv")
df_reviews = pd.read_csv("C:/python/python_for_data_science/hackason/reviews.csv")

In [3]:
df_calender = pd.read_csv("C:/python/python_for_data_science/hackason/calendar.csv.gz")

In [6]:
df_calender.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,15526,2025-06-28,f,NaN,NaN,5,1125
1,15526,2025-06-29,t,NaN,NaN,5,1125
2,15526,2025-06-30,t,NaN,NaN,5,1125
3,15526,2025-07-01,t,NaN,NaN,5,1125
4,15526,2025-07-02,t,NaN,NaN,5,1125
